# ToDo
- [x] Session-parallel mini-batch generation, where each session lengths is tracked
- [ ] (Optional) Add input connections to multiple GRU layers
- [x] Make the Loss Function Compatible with the Session-parallel mini-batch generation step
- [x] Output Sampling
- [x] Test Code(making predictions)
- [x] Check Loss Function
- [x] Check Optimizer
- [x] Check GRU layer
- [x] Check Evaluation code

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

dataset_root = Path('../sess-rec-large')
train = 'rsc15_train_full.txt'
test = 'rsc15_test.txt'
PATH_TO_TRAIN = dataset_root / train
PATH_TO_TEST = dataset_root / test

df_train = pd.read_csv(PATH_TO_TRAIN, sep='\t', dtype={'ItemId': np.int64})
df_test = pd.read_csv(PATH_TO_TEST, sep='\t', dtype={'ItemId': np.int64})

# Train

In [ ]:
from modules.model import GRU4REC
import torch

session_key = 'SessionId'
time_key = 'Time'
item_key = 'ItemId'

input_size = df_train[item_key].nunique()
output_size = input_size
hidden_size = 100
num_layers = 1
use_cuda = True
time_sort = False

dropout_hidden = .5
dropout_input = 0
batch_size = 50

torch.manual_seed(7)
torch.cuda.manual_seed(7)

model = GRU4REC(input_size, hidden_size, output_size,
                num_layers = num_layers,
                dropout_input = 0, dropout_hidden = dropout_hidden,
                batch_size = batch_size,
                use_cuda = use_cuda,
                loss_type = 'TOP1', optimizer_type = 'Adagrad', lr=.01,
                time_sort = time_sort)

model.train(df_train, session_key, time_key, item_key, n_epochs = 10)

# Evaluate

In [ ]:
import pickle
from pathlib import Path
import pandas as pd
import numpy as np

model_root = Path('./models')
model_name = 'GRU4REC_TOP1_Adagrad_0.05_epoch1'
model_file = model_root/model_name

dataset_root = Path('../sess-rec-large')
train = 'rsc15_train_full.txt'
test = 'rsc15_test.txt'
PATH_TO_TRAIN = dataset_root / train
PATH_TO_TEST = dataset_root / test

session_key = 'SessionId'
time_key = 'Time'
item_key = 'ItemId'

df_train = pd.read_csv(PATH_TO_TRAIN, sep='\t', dtype={'ItemId': np.int64})
df_test = pd.read_csv(PATH_TO_TEST, sep='\t', dtype={'ItemId': np.int64})

In [ ]:
from modules.layer import GRU
from modules.model import GRU4REC
import torch

input_size = df_train[item_key].nunique()
output_size = input_size
hidden_size = 100
num_layers = 1
use_cuda = True
time_sort = False

dropout_hidden = .5
dropout_input = 0
batch_size = 50

gru = GRU(input_size, hidden_size, output_size,
          num_layers = num_layers,
          dropout_input = dropout_input,
          dropout_hidden = dropout_hidden,
          batch_size = batch_size,
          use_cuda = use_cuda)

gru.load_state_dict(torch.load(model_file))

model = GRU4REC(input_size, hidden_size, output_size,
                num_layers = num_layers,
                dropout_input = 0, dropout_hidden = dropout_hidden,
                batch_size = batch_size,
                use_cuda = use_cuda,
                loss_type = 'TOP1', optimizer_type = 'Adagrad', lr=.01,
                time_sort = time_sort,
                pretrained = gru)

k = 20
recall, mrr = model.test(df_train, df_test, session_key,
                         time_key, item_key,
                         k = k, batch_size = 50)
result = f'Recall@{k}:{recall:.7f},MRR@{k}:{mrr:.7f}'
print(result)

In [5]:
!python run_train.py --n_samples 1000

Epoch: 1/Loss:0.994/TrainingTime:0.002(min)
Epoch: 2/Loss:0.968/TrainingTime:0.000(min)
Epoch: 3/Loss:0.955/TrainingTime:0.000(min)
Epoch: 4/Loss:0.950/TrainingTime:0.000(min)
Epoch: 5/Loss:0.947/TrainingTime:0.000(min)
Epoch: 6/Loss:0.945/TrainingTime:0.000(min)
Epoch: 7/Loss:0.944/TrainingTime:0.000(min)
Epoch: 8/Loss:0.943/TrainingTime:0.001(min)
Epoch: 9/Loss:0.943/TrainingTime:0.000(min)
Epoch:10/Loss:0.943/TrainingTime:0.000(min)
